<a href="https://colab.research.google.com/github/ssgalitsky/colab_utils/blob/master/caffe2py2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
print "Install deps and gcc"

with open('/etc/apt/sources.list') as f:
  txt = f.read()
with open('/etc/apt/sources.list', 'w') as f:
  f.write(txt.replace('# deb-src','deb-src'))
  
t=!apt update
t=!apt build-dep caffe-cuda
t=!apt install g++-5

t=!update-alternatives --remove-all gcc 
t=!update-alternatives --remove-all g++
t=!update-alternatives --install /usr/bin/gcc gcc /usr/bin/gcc-5 20
t=!update-alternatives --install /usr/bin/g++ g++ /usr/bin/g++-5 20
t=!update-alternatives --install /usr/bin/cc cc /usr/bin/gcc 30
t=!update-alternatives --set cc /usr/bin/gcc
t=!update-alternatives --install /usr/bin/c++ c++ /usr/bin/g++ 30
t=!update-alternatives --set c++ /usr/bin/g++

print "Get boost"

t=!wget https://dl.bintray.com/boostorg/release/1.67.0/source/boost_1_67_0.tar.bz2
t=!tar --bzip2 -xf boost_1_67_0.tar.bz2

print "Compile boost"

t=!cd boost_1_67_0 && ./bootstrap.sh --exec-prefix=/usr/local --with-libraries=system,filesystem,regex,thread,python \
--with-python-version=2.7 --with-python-root=/usr
t=!cd boost_1_67_0 && ./b2 install

print "Clone SSD-Caffe and configure it"

t=!git clone https://github.com/weiliu89/caffe.git && cd caffe && git checkout ssd
  
with open('caffe/Makefile.config.example') as f:
  config = f.read()
comment = ['CUDA_DIR := /usr/local/cuda', 
           'BLAS := open']
uncomment = ['# CUDA_DIR := /usr', 
             '# BLAS := atlas', 
             '# OPENCV_VERSION := 3', '# WITH_PYTHON_LAYER := 1'] #
replace = [('INCLUDE_DIRS := $(PYTHON_INCLUDE) /usr/local/include',
            'INCLUDE_DIRS := $(PYTHON_INCLUDE) /usr/local/include /usr/include/hdf5/serial /usr/local/lib/python2.7/dist-packages/numpy/core/include/'), 
           ('LIBRARY_DIRS := $(PYTHON_LIB) /usr/local/lib /usr/lib',
            'LIBRARY_DIRS := $(PYTHON_LIB) /usr/local/lib /usr/lib /usr/lib/x86_64-linux-gnu/hdf5/serial')]
for c in uncomment:
  config = config.replace(c, c[2:])
for c in comment:
  config = config.replace(c, '# '+c)
for c1,c2 in replace:
  config = config.replace(c1, c2)
with open('caffe/Makefile.config', 'w') as f:
  f.write(config)
  
with open('caffe/Makefile') as f:
  mfile = f.read()
with open('caffe/Makefile', 'w') as f:
  f.write(mfile.replace('-isystem','-I'))
  
with open('/usr/include/x86_64-linux-gnu/c++/5/bits/c++config.h') as f:
  txt = f.read()
with open('/usr/include/x86_64-linux-gnu/c++/5/bits/c++config.h', 'w') as f:
  f.write(txt.replace('/* #undef _GLIBCXX_USE_C99_MATH */',
                      '/* #undef _GLIBCXX_USE_C99_MATH */\n#define  _GLIBCXX_USE_C99_MATH  1'))
  
print "Make Caffe"

#t=!cd caffe && make -j8 && make pycaffe && make test -j8 && make distribute
t=!cd caffe && make -j8 && make pycaffe && make distribute
print '\n'.join(t[-10:])

t=!echo /usr/local/lib >> /etc/ld.so.conf && ldconfig
t=!echo /content/caffe/distribute/lib >> /etc/ld.so.conf && ldconfig

print "DONE"

Install deps and gcc
Get boost
Compile boost
Clone SSD-Caffe and configure it
Make Caffe
CXX src/caffe/util/db_lmdb.cpp
CXX src/caffe/util/bbox_util.cpp
CXX src/caffe/util/signal_handler.cpp
CXX src/caffe/util/sampler.cpp
CXX src/caffe/util/im_transforms.cpp
NVCC src/caffe/layers/exp_layer.cu
nvcc fatal   : Unsupported gpu architecture 'compute_20'
Makefile:588: recipe for target '.build_release/cuda/src/caffe/layers/exp_layer.o' failed
make: *** [.build_release/cuda/src/caffe/layers/exp_layer.o] Error 1
make: *** Waiting for unfinished jobs....
DONE
